In [1]:
# --- Imports ---

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

import pandas as pd
import numpy as np

from tqdm import tqdm
import time


In [2]:
# --- Global Variables and Hyperparameters ---
base_path = './kaggle/input/nfl-big-data-bowl-2026-prediction/train'
lr = .001
num_epochs = 50
batch_size = 64
num_weeks = 1
load_prev_model = False

In [3]:
# --- ETL Helpers ---

# Input: Array of input dataframes
# Output: Dictionary of dataframes grouped by id
def format_input(input):
    ipt = pd.concat(input, ignore_index=True) # Concatenate all dfs into one
    ipt = ipt[ipt['player_to_predict'] == True].copy(deep=True) # Filter to only players that matter

    # Add velocity component column
    dir_rad = np.deg2rad(ipt['dir'])
    ipt['vx'] = ipt['s'] * np.sin(dir_rad)
    ipt['vy'] = ipt['s'] * np.cos(dir_rad)

    # Add orientation components
    o_rad = np.deg2rad(ipt['o'])
    ipt['ox'] = np.sin(o_rad)
    ipt['oy'] = np.cos(o_rad)

    # Offense going left? 1 or -1
    ipt['go_left'] = np.where(ipt['play_direction'] == 'left', 1, -1)

    # Offensive player? 1 or -1
    ipt['offensive_player'] = np.where(ipt['player_role'] == 'Targeted Receiver', 1, -1)

    # Get useful columns only
    # Constant variables: 'go_left', 'offensive_player', 'ball_land_x', 'ball_land_y', 'absolute_yardline_number', 'num_frames_output'
    # Time variables : 'x', 'y', 'vx', 'vy', 'ox', 'oy', 'a', 'frame_id'
    ipt = ipt[['game_id', 'play_id', 'nfl_id', 'x', 'y', 'vx', 'vy', 'ox', 'oy', 'go_left', 'offensive_player', 'a', 'ball_land_x', 'ball_land_y', 'absolute_yardline_number', 'frame_id', 'num_frames_output']]


    # Create dictionary of dfs
    ipt = {
        f"{gid}_{pid}_{nid}": g
        for (gid, pid, nid), g in ipt.groupby(['game_id', 'play_id', 'nfl_id'])
    }
    return ipt


# Input: 
    # out: Target dataframe (use only to extract ids and get frame id)
    # ipt: Dictionary of input dfs
# Output: List of (x1, x2, y)
    # x1: First input vector
    # x2: Second input vector
    # x3: Target vector
def preprocess(out, ipt):
    data = []
    for _, row in out.iterrows():
        fid = f"{int(row['game_id'])}_{int(row['play_id'])}_{int(row['nfl_id'])}"
        instance = ipt[fid]
        frame = row['frame_id']
        
        # First Input Vector
        x0 = instance.iloc[-1][['go_left', 'offensive_player', 'ball_land_x', 'ball_land_y', 'absolute_yardline_number', 'num_frames_output']].values
        x1a = instance.iloc[-1][['x','y','vx','vy','ox','oy','a','frame_id']].values
        x1b = instance.iloc[-2][['x','y','vx','vy','ox','oy','a','frame_id']].values
        x1 = np.concatenate([x0, x1a, x1b, [frame]]).astype(np.float32)

        x2 = instance.iloc[-1][['x','y']].values.astype(np.float32) # Second Input Vector
        y  = row[['x','y']].values.astype(np.float32) # Target Vector

        data.append((x1, x2, y))
    return data

# Custom Dataset Class
class MyDataset(Dataset):
    def __init__(self, dat):
        self.data = dat

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x1, x2, y = self.data[idx]
        return torch.tensor(x1), torch.tensor(x2), torch.tensor(y)


Train and Test

In [5]:
# --- Train & Test Data ETL ---

print("Started loading training data")
raw_input_train = [pd.read_csv(f'{base_path}/input_2023_w0{i}.csv') for i in range(1,num_weeks+1)]
raw_output_train = [pd.read_csv(f'{base_path}/output_2023_w0{i}.csv') for i in range(1,num_weeks+1)]
in_train = format_input(raw_input_train)
out_train = pd.concat(raw_output_train, ignore_index=True)
train_dataset = MyDataset(preprocess(out_train, in_train))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
print("Finished loading training data")

print("Started loading testing data")
raw_input_test = [pd.read_csv(f'{base_path}/input_2023_w{i}.csv') for i in range(16,19)]
raw_output_test = [pd.read_csv(f'{base_path}/output_2023_w{i}.csv') for i in range(16,19)]
in_test = format_input(raw_input_test)
out_test = pd.concat(raw_output_test, ignore_index=True)
test_dataset = MyDataset(preprocess(out_test, in_test))
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
print("Finished loading testing data")

Started loading training data
Finished loading training data
Started loading testing data
Finished loading testing data


In [9]:
# --- Model Initialization ---

class MLP(nn.Module):
    def __init__(self):
        super().__init__()

        self.mlp = nn.Sequential(
            nn.Linear(23, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )

    def forward(self, x1, x2):
        y = self.mlp(x1)
        out = x2 + y
        return out
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load("V3.pth", weights_only=False, map_location=device) if load_prev_model else torch.compile(MLP().to(device))
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss(reduction="mean")

In [10]:
# --- Training Loop ---
for epoch in range(num_epochs):
    total_loss = 0.0
    
    for x1, x2, y in train_loader:
        x1, x2, y = x1.to(device, non_blocking=True), x2.to(device, non_blocking=True), y.to(device, non_blocking=True)

        preds = model(x1, x2)
        loss = torch.sqrt(criterion(preds, y))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss = {total_loss / len(train_loader):.4f}")

# torch.save(model, "V3.pth")

Epoch 1: Loss = 2.1971
Epoch 2: Loss = 1.4325
Epoch 3: Loss = 1.3397
Epoch 4: Loss = 1.2824
Epoch 5: Loss = 1.2138
Epoch 6: Loss = 1.1801
Epoch 7: Loss = 1.1276
Epoch 8: Loss = 1.0965
Epoch 9: Loss = 1.0445
Epoch 10: Loss = 1.0281
Epoch 11: Loss = 0.9885
Epoch 12: Loss = 0.9334
Epoch 13: Loss = 0.9122
Epoch 14: Loss = 0.8668
Epoch 15: Loss = 0.8508
Epoch 16: Loss = 0.8222
Epoch 17: Loss = 0.7957
Epoch 18: Loss = 0.7596
Epoch 19: Loss = 0.7461
Epoch 20: Loss = 0.7489
Epoch 21: Loss = 0.7255
Epoch 22: Loss = 0.6993
Epoch 23: Loss = 0.6934
Epoch 24: Loss = 0.6633
Epoch 25: Loss = 0.6660
Epoch 26: Loss = 0.6536
Epoch 27: Loss = 0.6398
Epoch 28: Loss = 0.6347
Epoch 29: Loss = 0.6252
Epoch 30: Loss = 0.6111
Epoch 31: Loss = 0.6139
Epoch 32: Loss = 0.5878
Epoch 33: Loss = 0.5791
Epoch 34: Loss = 0.5741
Epoch 35: Loss = 0.5696
Epoch 36: Loss = 0.5592
Epoch 37: Loss = 0.5555
Epoch 38: Loss = 0.5557
Epoch 39: Loss = 0.5339
Epoch 40: Loss = 0.5397
Epoch 41: Loss = 0.5395
Epoch 42: Loss = 0.5283
E

In [11]:
# --- Test model ---
model.eval()

with torch.no_grad():
    total_loss = 0.0
    for x1, x2, y in tqdm(train_loader, leave=False):
        x1, x2, y = x1.to(device, non_blocking=True), x2.to(device, non_blocking=True), y.to(device, non_blocking=True)
        preds = model(x1, x2)
        loss = torch.sqrt(criterion(preds, y))
        total_loss += loss.item()
    loss = total_loss / len(train_loader)
    print(f"Loss = {loss}")

Loss = 0.5603106991347089


Submission

In [8]:
# --- Submission data preprocessing ---
instances = pd.read_csv('./kaggle/input/nfl-big-data-bowl-2026-prediction/test.csv')
eval_in = pd.read_csv('./kaggle/input/nfl-big-data-bowl-2026-prediction/test_input.csv')
eval_in = format_input([eval_in])

def compute_pos(row):
    gid, pid, nid, fid = row['game_id'], row['play_id'], row['nfl_id'], row['frame_id']
    full_id = f"{gid}_{pid}_{nid}_{fid}"

    x1, x2 = prepare_inputs(row, eval_in)
    x1, x2 = x1.to(device, non_blocking=True), x2.to(device, non_blocking=True)

    pred_x, pred_y = model(x1, x2).squeeze(0).tolist()
    return pd.Series([full_id, pred_x, pred_y])

In [9]:
# --- Final submission df ---
model.eval()
submission = instances.copy(deep=True)

with torch.no_grad():
    submission[['id', 'x', 'y']] = submission.apply(compute_pos, axis=1)
submission = submission[['id', 'x', 'y']]

submission.head(10)

,id,x,y
0,2024120805_74_54586_1,88.500992,34.264431
1,2024120805_74_54586_2,88.512939,34.281620
2,2024120805_74_54586_3,88.577324,34.344898
3,2024120805_74_54586_4,88.890472,34.674427
4,2024120805_74_54586_5,89.433800,34.718723
5,2024120805_74_54586_6,89.643997,34.759796
6,2024120805_74_54586_7,89.694168,34.883747
7,2024120805_74_54586_8,89.743332,35.010021
8,2024120805_74_54586_9,89.892632,35.121971
9,2024120805_74_54586_10,90.063416,35.226063


In [10]:
# --- Submission to csv ---
submission.to_csv('submission.csv', index=False)